In [ ]:
from Classes.Person import Person
import pickle
import numpy as np
import math
import pandas as pd
import seaborn as sns

In [ ]:
# honest = pickle.load(open("batch_run_honest_1","rb"))
# strategic = pickle.load(open("batch_run_strategic_1","rb"))
test = pickle.load(open("batch_run_strategic_test","rb"))

In [ ]:
test.head()

In [ ]:
def convertRuns(_data):
    runs = pd.DataFrame(columns=['run_id', 'cand_unique_id', 'cand_votes', 'cand_position'])
    for i, run in _data.iterrows():
        for cand in run[1][0]:
            runs = runs.append({'run_id':i,'cand_unique_id':cand.unique_id,'cand_votes':run[1][0].get(cand),'cand_position':cand.position},ignore_index=True)
    runs['cand_votes'] = runs['cand_votes'].convert_dtypes(int)
    return runs

In [ ]:
# honest_runs = convertRuns(honest) 
# honest_runs.head()

In [ ]:
# strategic_runs = convertRuns(strategic) 
# strategic_runs.head()

In [ ]:
test_runs = convertRuns(test)
test_runs.head()

In [ ]:
def addDistance(df:pd.DataFrame):
    df['distance'] = df['cand_position'].apply(lambda x: np.linalg.norm([1.0,1.0] - x))
#     df['winperc'] = df['cand_votes'].apply(lambda x: x/1000)
    return df
        

In [ ]:
# honest_runs = addDistance(honest_runs)
# display(honest_runs.sort_values(by=['distance']))
# display(honest_runs.sort_values(by=['cand_votes'],ascending=False))

# strategic_runs = addDistance(strategic_runs)
# display(strategic_runs.sort_values(by=['distance']))
# display(strategic_runs.sort_values(by=['cand_votes'],ascending=False))

test_runs = addDistance(test_runs)
display(test_runs.sort_values(by=['distance']))
display(test_runs.sort_values(by=['cand_votes'],ascending=False))

In [ ]:
# display(sns.pairplot(honest_runs[['cand_votes','distance']]))
# display(sns.pairplot(strategic_runs[['cand_votes','distance']]))

In [ ]:
# display(honest_runs[['cand_votes']].describe(include='all'))
# display(honest_runs[['cand_votes']].plot.box())
# display(strategic_runs[['cand_votes']].describe(include='all'))
# display(strategic_runs[['cand_votes']].plot.box())
display(test_runs[['cand_votes']].plot.box())

In [ ]:
display(test_runs[['distance']].plot.box())